In [51]:
# Import pandas and numpy
import pandas as pd
import numpy as np

In [11]:
# Read in the entire .tsv, with chunk size equal t0 100000
national = pd.read_csv('/Users/lin-chuntseng/Desktop/arcos_all_washpost.tsv', sep='\t', chunksize = 100000 , low_memory=False)

# Extract rows that are from the chosen states
keepers = list()
states = ['GA', 'AL', 'MS', 'SC', 'LA', 'NC', 'AR','FL']
for i in national:
    test = i['BUYER_STATE'].isin(states)
    rows_to_keep = i.loc[test]
    keepers.append(rows_to_keep)
new_df = pd.concat(keepers)

In [93]:
# Add a YEAR column by extracting the last 4 digit from TRANSACTION_DATE
new_df['YEAR'] = new_df['TRANSACTION_DATE'].astype('str').str[-4:]

In [96]:
# Add a Total_Weight column that is the multiple of MME_Conversion_Factor and CALC_BASE_WT_IN_GM
new_df['Total_Weight'] = new_df['MME_Conversion_Factor'] * new_df['CALC_BASE_WT_IN_GM']

In [206]:
# Keep necessary columns for the dataframe
necessary_col = ['BUYER_COUNTY','BUYER_STATE', 'YEAR','Total_Weight']

# Rename the trimmed new_df as prescription
prescription = new_df[necessary_col]


In [207]:
# We have 130 missing counties
prescription.isnull().sum()

BUYER_COUNTY    130
BUYER_STATE       0
YEAR              0
Total_Weight      0
dtype: int64

In [208]:
# Drop missing counties and check again
prescription = prescription.dropna()
prescription.isnull().sum()

BUYER_COUNTY    0
BUYER_STATE     0
YEAR            0
Total_Weight    0
dtype: int64

In [209]:
# GroupBy State, county, year
prescription = prescription.groupby(['BUYER_STATE','BUYER_COUNTY','YEAR'], as_index=False).sum()

In [216]:
prescription.head()

,BUYER_STATE,BUYER_COUNTY,YEAR,Total_Weight
0,AL,AUTAUGA,2006,17100.542225
1,AL,AUTAUGA,2007,19152.437289
2,AL,AUTAUGA,2008,21785.835290
3,AL,AUTAUGA,2009,24439.987693
4,AL,AUTAUGA,2010,26169.308158


In [215]:
# Export prescription to a .csv for later use
prescription.to_csv(r'/Users/lin-chuntseng/Desktop/2006-2012_prescription.csv', index = False, header = True)